# 7144COMP/CW2: Bird Multiple Object Detection Using Faster R-CNN ResNet101 Network 
## PART IV: Model evaluation and deployment

### Overview

In this notebook, I will evaluate my model through TensorBoard while using the generated metrics to determine model convergence (both validation loss and Intersection over Union (IoU) at both 0.5 and 0.75 are considered). 

The number of epochs to train the model is set to 1, the reason for this choice was explained in the training notebook. In addition, during the 1st epoch of training, the model converged around the final loss value (smoothed loss value with a weight of 0.8).



For the current task, the following steps have been undertaken: 

- Launch TensorBoard displaying both the train and evaluation metrics for the given session. 
- Provide justification for the number of epochs used for training your object detection model

### Next

In the next notebook which is an extension to the present, I will:

- Freeze my trained model in correct format for model inferencing
- Develop a Jupyter Notebook to perform inference on the frozen model using unseen test images
- Discuss my results.

### Prerequisites
This notebook runs locally on the environment *tf-gpu*.
- Environment Setup (see Part 0)
- Preprocessing (see Part 1)
- Training (see Part 2)
- Run the necessary evaluation scripts (see Part 3)

## 1. Import the necessary packages

In [1]:
import os

In [2]:
# Current directory
current_dir = os.getcwd()

# Model training directory and config pipeline
model_dir = os.path.join(current_dir, 'training')
pipeline_config_path = 'fasterrcnn_config.config'

## 2. TensorBoard 
### 2.1. Monitor region proposal losses, evaluation metrics
Here ```logdir``` points to the training directory, by launching the next cell, different loss graphs for region proposal network will be imported by TensorBoard from ```training/train``` folder, whereas evaluation metrics for the given session will be imported from the ```training/eval``` folder.

The losses for the Region Proposal Network:

- ```Loss/RPNLoss/localization_loss```: Localization Loss or the Loss of the Bounding Box regressor for the RPN

- ```Loss/RPNLoss/objectness_loss```: Loss of the Classifier that classifies if a bounding box is an object of interest or background

The losses for the Final Classifier:

- ```Loss/BoxClassifierLoss/classification_loss```: Loss for the classification of detected objects into various classes: Cat, Dog, Airplane etc

- ```BoxClassifierLoss/localization_loss```: Localization Loss or the Loss of the Bounding Box regressor



### Display the train and evaluation metrics for the given session 


In [3]:
!tensorboard --logdir $current_dir'/training/'

2023-01-01 20:36:36.459974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 20:36:37.108106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2023-01-01 20:36:37.108157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2023-01-01 20:36:37.108166: W tensorflow/compiler/tf2tensorrt/utils/py_utils

## 3. Discussion

### Training vs Validation losses

- The training total loss was ```train_total_loss = 0.1784```, whereas the validation total loss was ```val_total_loss = 0.2659``` at the last step of training. 
- Both losses seem to converge down towards lower error levels, which is a good sign that the model is learning.


### Precision Metrics


<table>
	<thead>
		<tr>
			<th>Metric</th>
			<th>Value</th>
			<th>Interpretation</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td>Average Precision (AP)</td>
			<td>0.549</td>
			<td>The model&#39;s average precision across all object sizes and IoU thresholds is 0.549</td>
		</tr>
		<tr>
			<td>Average Precision (AP) @ IoU=0.50</td>
			<td>0.871</td>
			<td>The model&#39;s average precision with an IoU threshold of 0.50 is 0.871</td>
		</tr>
		<tr>
			<td>Average Precision (AP) @ IoU=0.75</td>
			<td>0.642</td>
			<td>The model&#39;s average precision with an IoU threshold of 0.75 is 0.642</td>
		</tr>
		<tr>
			<td>Average Precision (AP) (small)</td>
			<td>0.177</td>
			<td>The model&#39;s average precision for small objects is 0.177</td>
		</tr>
		<tr>
			<td>Average Precision (AP) (medium)</td>
			<td>0.419</td>
			<td>The model&#39;s average precision for medium-sized objects is 0.419</td>
		</tr>
		<tr>
			<td>Average Precision (AP) (large)</td>
			<td>0.623</td>
			<td>The model&#39;s average precision for large objects is 0.623</td>
		</tr>
		<tr>
			<td>Average Recall (AR) @ maxDets=1</td>
			<td>0.604</td>
			<td>The model&#39;s average recall with a maximum of 1 detection per image is 0.604</td>
		</tr>
		<tr>
			<td>Average Recall (AR) @ maxDets=10</td>
			<td>0.651</td>
			<td>The model&#39;s average recall with a maximum of 10 detections per image is 0.651</td>
		</tr>
		<tr>
			<td>Average Recall (AR) @ maxDets=100</td>
			<td>0.654</td>
			<td>The model&#39;s average recall with a maximum of 100 detections per image is 0.654</td>
		</tr>
		<tr>
			<td>Average Recall (AR) (small)</td>
			<td>0.244</td>
			<td>The model&#39;s average recall for small objects is 0.244</td>
		</tr>
		<tr>
			<td>Average Recall (AR) (medium)</td>
			<td>0.555</td>
			<td>The model&#39;s average recall for medium-sized objects is 0.555</td>
		</tr>
		<tr>
			<td>Average Recall (AR) (large)</td>
			<td>0.714</td>
			<td>The model&#39;s average recall for large objects is 0.714</td>
		</tr>
	</tbody>
</table>


- The model's performance for object detection is average, with an average precision of ```0.549``` and an average recall of ```0.654```. The average precision indicates the model's ability to correctly identify objects and avoid false positives, while the average recall indicates its ability to detect all instances of the objects in the image.

- The model performs particularly well in terms of precision for objects with an ```IoU=0.50``` (intersection over union), which means that the predicted bounding box for the object has a ```50%``` overlap with the ground truth bounding box. The model's precision for this IoU threshold is ```0.871```. However, the model performs less well for objects with an ```IoU=0.75```, with a precision of ```0.642```.

- In terms of recall, the model performs better for larger objects, with a value of ```0.714``` compared to ```0.244``` for small objects. This means that the model is more successful at detecting large objects in the image, although there is still room for improvement in detecting small objects.

### Justification for the number of epochs used for training your object detection model

```num_epochs``` : A number of epochs of ```1``` means that the model will only make one pass through the entire training dataset. Since we'd like to quickly test the performance of the model on a small training data. However, for most real-world datasets, it is generally not sufficient to train a model to good performance, as the model will not have the opportunity to learn from the entire dataset.

While it is generally not recommended to train an object detection model using only one epoch, our purpose is to test the model's ```pretrained weights``` used as a starting point in training and our ```hyperparameters config``` on a relatively small dataset of 4000 images. This is due to the limitation in time and resources.

Increasing the number of epochs (num_epochs) during training can help improve the performance of an object detection model in a few different ways:

- **Increased model convergence**: by allowing the model to see the training data more times, which can help it learn more effectively and converge on a better solution.

- **Improved generalisation**: A model that has been trained for more epochs may be better able to generalize to new, unseen data. This is because the model has been exposed to more diverse examples during training and has had more opportunities to learn about the underlying patterns in the data.

- **More time for optimisation**: Training for more epochs gives the model more time to adjust its weights and biases through the optimization process. This can lead to improved model performance, especially if the learning rate is set appropriately.

Nevertheless, increasing the number of epochs also increases the training time and can lead to overfitting if the model is trained for too many epochs. Overfitting occurs when a model becomes *too specialized to the training data and performs poorly on new, unseen data*.

Note: It was not possible to increase the batch size due to memory limitations. 

#### **Comparison with Roboflow Cloud-based experiment (using the same model and dataset)**

- Using ```num_epochs=40``` and ```batch_size=50```, same augmentation steps and the same hyperparameters the model achieved ```90.1% mAP``` ```88.4% precision``` and ```82.8% recall``` with an average class ```precision``` of ```90%``` on the validation dataset.

- We can conclude that increasing the number of epochs and the batch size would lead to better precision and faster training.

<img src="https://storage.googleapis.com/roboflow-platform-cache/RjBpFWbVLQdI2NaOrqg24Eooatr2/qYHiTyjFVuJ6MWIK56Sh/4/results.png" width="800" />

### Next

- Freeze the trained model in correct format for model inferencing
- Develop a Jupyter Notebook to perform inference on the frozen model using unseen test images
- Discuss my results.